[View in Colaboratory](https://colab.research.google.com/github/hanshika0292/Deep-Learning-Assignments/blob/master/HANSHIKA_BATCH_4_ASSIGNMENT4B.ipynb)

In [27]:
# https://keras.io/
!pip install -q keras
import keras

In [28]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.optimizers import Adam

In [29]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [30]:
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 50
l = 250
num_filter = 24
compression = 0.5
dropout_rate = 0.2
NUM_TO_AUGMENT=5

In [31]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [32]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [33]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [34]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [35]:
num_filter = 24
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [36]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_53 (Conv2D)              (None, 32, 32, 24)   648         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_53 (BatchNo (None, 32, 32, 24)   96          conv2d_53[0][0]                  
__________________________________________________________________________________________________
activation_53 (Activation)      (None, 32, 32, 24)   0           batch_normalization_53[0][0]     
__________________________________________________________________________________________________
conv2d_54 

In [37]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [43]:
print("Augmenting training set images...")
datagen = ImageDataGenerator(
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
fill_mode='nearest')
test_gen = ImageDataGenerator()

Augmenting training set images...


In [45]:
train_generator = datagen.flow(x_train, y_train, batch_size=64)
test_generator = test_gen.flow(x_test, y_test, batch_size=64)

In [46]:
model.fit_generator(train_generator, steps_per_epoch=50000//64, epochs=10, validation_data=test_generator, validation_steps=10000//64)

Epoch 1/10
781/781 [==============================] - 361s 463ms/step - loss: 0.8137 - acc: 0.7199 - val_loss: 0.6790 - val_acc: 0.7967
Epoch 2/10
123/781 [===>..........................] - ETA: 4:49 - loss: 0.7327 - acc: 0.7444

781/781 [==============================] - 360s 461ms/step - loss: 0.7383 - acc: 0.7419 - val_loss: 0.8361 - val_acc: 0.7694
Epoch 3/10
214/781 [=======>......................] - ETA: 4:09 - loss: 0.7225 - acc: 0.7498

781/781 [==============================] - 360s 461ms/step - loss: 0.7177 - acc: 0.7503 - val_loss: 0.6831 - val_acc: 0.7919
Epoch 4/10
248/781 [========>.....................] - ETA: 3:55 - loss: 0.7014 - acc: 0.7588

781/781 [==============================] - 361s 462ms/step - loss: 0.7027 - acc: 0.7567 - val_loss: 0.6697 - val_acc: 0.7990
Epoch 5/10
261/781 [=========>....................] - ETA: 3:49 - loss: 0.7119 - acc: 0.7523

781/781 [==============================] - 361s 462ms/step - loss: 0.6975 - acc: 0.7582 - val_loss: 0.6930 - val_acc: 0.7865
Epoch 6/10
266/781 [=========>....................] - ETA: 3:46 - loss: 0.6960 - acc: 0.7625

781/781 [==============================] - 360s 462ms/step - loss: 0.6820 - acc: 0.7643 - val_loss: 0.7143 - val_acc: 0.7936
Epoch 7/10
268/781 [=========>....................] - ETA: 3:44 - loss: 0.6766 - acc: 0.7664

781/781 [==============================] - 359s 460ms/step - loss: 0.6763 - acc: 0.7644 - val_loss: 0.7408 - val_acc: 0.7800
Epoch 8/10
268/781 [=========>....................] - ETA: 3:45 - loss: 0.6608 - acc: 0.7691

781/781 [==============================] - 360s 461ms/step - loss: 0.6660 - acc: 0.7672 - val_loss: 0.5903 - val_acc: 0.8151
Epoch 9/10
268/781 [=========>....................] - ETA: 3:44 - loss: 0.6650 - acc: 0.7693

781/781 [==============================] - 359s 460ms/step - loss: 0.6641 - acc: 0.7702 - val_loss: 1.1713 - val_acc: 0.7113
Epoch 10/10
267/781 [=========>....................] - ETA: 3:45 - loss: 0.6638 - acc: 0.7707

781/781 [==============================] - 360s 461ms/step - loss: 0.6583 - acc: 0.7721 - val_loss: 0.5860 - val_acc: 0.8229


In [50]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=37,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/17
43776/50000 [=========================>....] - ETA: 40s - loss: 0.1896 - acc: 0.9334

50000/50000 [==============================] - 343s 7ms/step - loss: 0.1929 - acc: 0.9321 - val_loss: 0.7291 - val_acc: 0.8166
Epoch 2/17
26240/50000 [==============>...............] - ETA: 2:36 - loss: 0.1859 - acc: 0.9340

50000/50000 [==============================] - 347s 7ms/step - loss: 0.1880 - acc: 0.9335 - val_loss: 0.5344 - val_acc: 0.8597
Epoch 3/17
19584/50000 [==========>...................] - ETA: 3:20 - loss: 0.1841 - acc: 0.9328

50000/50000 [==============================] - 347s 7ms/step - loss: 0.1887 - acc: 0.9314 - val_loss: 0.5057 - val_acc: 0.8666
Epoch 4/17
17024/50000 [=========>....................] - ETA: 3:37 - loss: 0.1815 - acc: 0.9362

50000/50000 [==============================] - 347s 7ms/step - loss: 0.1832 - acc: 0.9357 - val_loss: 0.5645 - val_acc: 0.8533
Epoch 5/17
16064/50000 [========>.....................] - ETA: 3:44 - loss: 0.1714 - acc: 0.9376

...

50000/50000 [==============================] - 347s 7ms/step - loss: 0.1649 - acc: 0.9409 - val_loss: 0.5151 - val_acc: 0.8697
Epoch 14/17
15360/50000 [========>.....................] - ETA: 3:48 - loss: 0.1575 - acc: 0.9410

50000/50000 [==============================] - 347s 7ms/step - loss: 0.1584 - acc: 0.9427 - val_loss: 0.5560 - val_acc: 0.8677
Epoch 15/17
15360/50000 [========>.....................] - ETA: 3:48 - loss: 0.1533 - acc: 0.9460

50000/50000 [==============================] - 348s 7ms/step - loss: 0.1599 - acc: 0.9432 - val_loss: 0.8216 - val_acc: 0.8205
Epoch 16/17
15360/50000 [========>.....................] - ETA: 3:49 - loss: 0.1516 - acc: 0.9462

50000/50000 [==============================] - 348s 7ms/step - loss: 0.1563 - acc: 0.9447 - val_loss: 0.4744 - val_acc: 0.8800
Epoch 17/17
15360/50000 [========>.....................] - ETA: 3:49 - loss: 0.1470 - acc: 0.9461

50000/50000 [==============================] - 347s 7ms/step - loss: 0.1540 - acc: 0.9449 - val_loss: 0.4607 - val_acc: 0.8804


In [51]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 22s 2ms/step
Test loss: 0.4606768587812781
Test accuracy: 0.8804


In [52]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [53]:
from google.colab import files

files.download('DNST_model.h5')